## Data Analysis for EPL Predictor
We format the premier league data sourced publicly from https://www.football-data.co.uk/englandm.php by adding rankings for each team based on previous seasons 

In [21]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import copy

## Exploratory Data Analysis 
We analyze our dataset and understand the relationships between different features.

In [22]:
def loadDict(data):
    # load the data
    df = pd.read_csv(data, na_filter=True, on_bad_lines='skip', encoding='utf-8')
    refCols = list(["Div","Date","HomeTeam","AwayTeam","FTHG","HG","FTAG","AG","FTR","Res","HTHG","HTAG","HTR","Attendance","Referee","HS","AS","HST","AST","HHW","AHW","HC","AC","HF","AF","HFKC","AFKC","HO","AO","HY","AY","HR","AR","HBP","ABP"])
    cols = list(df.columns)
    dropCols = (set(cols) - set(refCols))
    df.drop(dropCols, axis=1, inplace=True)
    return df

#initialise the dictionary
raw_data = {}
os.chdir("/Users/tanmaypilla/Desktop/prempredictor/data")
for i,j in enumerate(os.listdir()):
    print("Loading: ", j)
    raw_data[i] = loadDict(j)
print("The following years were loaded (yy): ", raw_data.keys())



Loading:  2005_2006.csv
Loading:  2013_2014.csv
Loading:  2004_2005.csv
Loading:  2022_2023.csv
Loading:  2012_2013.csv
Loading:  2017_2018.csv
Loading:  2007_2008.csv
Loading:  2002_2003.csv
Loading:  2014_2015.csv
Loading:  2009_2010.csv
Loading:  2015_2016.csv
Loading:  2003_2004.csv
Loading:  2010_2011.csv
Loading:  2008_2009.csv
Loading:  2011_2012.csv
Loading:  2021_2022.csv
Loading:  2020_2021.csv
Loading:  2006_2007.csv
Loading:  2016_2017.csv
Loading:  2019_2020.csv
Loading:  2001_2002.csv
Loading:  2018_2019.csv
Loading:  2000_2001.csv
The following years were loaded (yy):  dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])


In [23]:
def create_HomeDict(table, matchNo):
    match = {}
    if any('FTR' == table.keys()):
        match['result'] = table['FTR'].values[0]
    elif any('Res' == table.keys()):
        match['result'] = table['Res'].values[0]
    if any('FTHG' == table.keys()):
        match['Goals'] = table['FTHG'].values[0]
    elif any('HG' == table.keys()):
        match['Goals'] = table['HG'].values[0]
    if any('FTAG' == table.keys()):
        match['GoalsConceded'] = table['FTAG'].values[0]
    elif any('AG' == table.keys()):
        match['GoalsConceded'] = table['AG'].values[0]
    match.update({'match' : matchNo, 'ground' : 'H', 'Date' : table['Date'].values[0], \
                   'TeamAgainst' : table['AwayTeam'].values[0], 'HTGoals' : table['HTHG'].values[0], \
                    'HTResult' : table['HTR'].values[0], 'Shots' :table['HS'].values[0], \
                    'ShotsOnTarget' : table['HST'].values[0], 'ShotsAgainst' : table['AS'].values[0], \
                    'ShotsAgainstOnTarget' : table['AST'].values[0], 'Corners' : table['HC'].values[0], \
                    'CornersAgainst' : table['AC'].values[0], 'FoulsCommitted' : table['HF'].values[0], \
                    'FoulsAgainst' : table['AF'].values[0], 'YellowCards' : table['HY'].values[0], \
                    'YellowCardsAgainst' : table['AY'].values[0], 'RedCards' : table['HR'].values[0], \
                    'RedCardsAgainst' : table['AR'].values[0]})
    match.update({'BigChancesCreated' : match['ShotsOnTarget'] + match['Goals']})
    if match['result'] == 'H':
        match.update({'Win' : 1, 'Draw' : 0, 'Loss' : 0})
    elif match['result'] == 'D':
        match.update({'Win' : 0, 'Draw' : 1, 'Loss' : 0})
    elif match['result'] == 'A':
        match.update({'Win' : 0, 'Draw' : 0, 'Loss' : 1})
    return pd.DataFrame(match, index=[matchNo,])

def create_AwayDict(table, matchNo):
    match = {}
    if any('FTR' == table.keys()):
        match['result'] = table['FTR'].values[0]
    elif any('Res' == table.keys()):
        match['result'] = table['Res'].values[0]
    if any('FTHG' == table.keys()):
        match['GoalsConceded'] = table['FTHG'].values[0]
    elif any('HG' == table.keys()):
        match['GoalsConceded'] = table['HG'].values[0]
    if any('FTAG' == table.keys()):
        match['Goals'] = table['FTAG'].values[0]
    elif any('AG' == table.keys()):
        match['Goals'] = table['AG'].values[0]
    match.update({'match' : matchNo, 'ground' : 'A', 'Date' : table['Date'].values[0], \
                'TeamAgainst' : table['HomeTeam'].values[0], 'HTGoals' : table['HTAG'].values[0], \
                'HTResult' :table['HTR'].values[0], 'Shots' : table['AS'].values[0], 'Shots Against' :table['HS'].values[0], \
                'ShotsOnTarget' : table['AST'].values[0], 'ShotsAgainstOnTarget' : table['HST'].values[0], \
                'Corners' : table['AC'].values[0], 'CornersAgainst' : table['HC'].values[0], 'FoulsCommitted' :table['AF'].values[0], \
                'FoulsAgainst' : table['HF'].values[0], 'YellowCards' : table['AY'].values[0], 'YellowCardsAgainst' : table['HY'].values[0], \
                'RedCards' : table['AR'].values[0], 'RedCardsAgainst' : table['HR'].values[0]})
    match.update({'BigChancesCreated' : match['ShotsOnTarget'] + match['Goals']})
    if match['result'] == 'A':
        match.update({'Win' : 1, 'Draw' : 0, 'Loss' : 0})
    elif match['result'] == 'D':
        match.update({'Win' : 0, 'Draw' : 1, 'Loss' : 0})
    elif match['result'] == 'H':
        match.update({'Win' : 0, 'Draw' : 0, 'Loss' : 1})

def build_snapshot_table(raw_data, snapshots):
    for i in raw_data.keys():
        snapshots[i] = {}
        for j in list(set(raw_data[i]['AwayTeam'])):
            snapshots[i][j] = pd.DataFrame()
            tempTable = raw_data[i][(raw_data[i]['AwayTeam']==j) | (raw_data[i]['HomeTeam']==j)]
            for k in range(len(tempTable)):
                if tempTable.iloc[k]['HomeTeam'] == j:
                    snapshots[i][j] = pd.concat([snapshots[i][j],create_HomeDict(tempTable[k:k+1], k+1)])
                elif tempTable.iloc[k]['AwayTeam'] == j:
                    snapshots[i][j] = pd.concat([snapshots[i][j],create_AwayDict(tempTable[k:k+1], k+1)])
    return snapshots

snapshots={}
snapshots = build_snapshot_table(raw_data,snapshots)
print("the number of seasons built into the snapshots are:")
print(snapshots.keys())
print("In season of 2010, the following teams played:")
print(snapshots[10].keys())
print("Liverpools games looked like this:")
print((snapshots[10]['Liverpool']).head(3))

the number of seasons built into the snapshots are:
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])
In season of 2010, the following teams played:
dict_keys(['Leicester', 'Middlesbrough', 'Swansea', 'West Ham', 'Liverpool', 'West Brom', 'Sunderland', 'Hull', 'Chelsea', 'Burnley', 'Watford', 'Man City', 'Crystal Palace', 'Arsenal', 'Everton', 'Southampton', 'Bournemouth', 'Stoke', 'Tottenham', 'Man United'])
Liverpools games looked like this:
  result  Goals  GoalsConceded  match ground      Date TeamAgainst  HTGoals  \
4      H      4              1      4      H  10/09/16   Leicester        2   
6      H      5              1      6      H  24/09/16        Hull        3   
8      D      0              0      8      H  17/10/16  Man United        0   

  HTResult  Shots  ...  FoulsCommitted  FoulsAgainst  YellowCards  \
4        H     17  ...               4             8            1   
6        H     32  ...               8             8 

In [24]:
def build_season_stats(teamDF,team):
    return {'Team': team,
    'Wins': sum(teamDF['Win']),
    'Losses': sum(teamDF['Loss']),
    'Draws': sum(teamDF['Draw']),
    'Goals': sum(teamDF['Goals']),
    'GoalsAgainst': sum(teamDF['GoalsConceded']),
    'YCards': sum(teamDF['YellowCards']),
    'RCards': sum(teamDF['RedCards']),
    'avg_Goals':np.mean(teamDF['Goals']),
    'avg_GoalsAgainst':np.mean(teamDF['GoalsConceded']),
    'avg_Corners':np.mean(teamDF['Corners']),
    'avg_CornersAgaints':np.mean(teamDF['CornersAgainst']),
    'avg_Fouls':np.mean(teamDF['FoulsCommitted']),
    'avg_FoulsAgainst':np.mean(teamDF['FoulsAgainst']),
    'avg_Shots':np.mean(teamDF['Shots']),
    'avg_ShotsAgainst':np.mean(teamDF['ShotsAgainst']),
    'avg_BigChancesCreated':np.mean(teamDF['BigChancesCreated']),
    }
## Take a season + team snapshot, query all,home and away games into smaller dataframes and feed them into build_season_stats,
## Add home and away prefix to show aggregations specific to the stadiums.

def build_team_summary(teamDF, team):
    if 'ground' not in teamDF.columns:
        raise ValueError("Column 'ground' not found in the DataFrame.")

    homeDF = teamDF.query('ground == "H" ', engine='python')
    awayDF = teamDF.query('ground == "A" ', engine='python')
    seasonDict = build_season_stats(teamDF, team)
    homeDict = build_season_stats(homeDF, team)
    awayDict = build_season_stats(awayDF, team)
    
    homeDict = dict(zip(["home_" + i for i in homeDict.keys()], homeDict.values()))
    awayDict = dict(zip(["away_" + i for i in awayDict.keys()], awayDict.values()))
    
    seasonDict = {**seasonDict, **homeDict, **awayDict}
    seasonDict = {**seasonDict, **{'Points': 3 * seasonDict['Wins'] + (1 * seasonDict['Draws']), 'GD': seasonDict['Goals'] - seasonDict['GoalsAgainst']}}

    return pd.DataFrame(seasonDict, index=[team,])


## Build entire database, take snapshots data and run through season+team data.
def build_season_table(snapshots):
    seasonTab = {}
    for i in snapshots.keys():
        seasonTab[i] = pd.DataFrame()
        for k in snapshots[i].keys():
            seasonTab[i] = pd.concat([seasonTab[i],(build_team_summary(snapshots[i][k],k))])
        seasonTab[i] = seasonTab[i].sort_values(by=['Points','GD'],ascending=False)
        seasonTab[i]['Position'] = np.linspace(1,20,20)
    return seasonTab

## build into seasonTable
seasonTable = build_season_table(snapshots)

print("The following seasons have been loaded")
print(seasonTable.keys())
print("This is what the 2010 season looked like")
print((seasonTable[10].sort_values(by=['Position'],ascending=True)).head())
print("These are the seasonal features available:")
print(seasonTable[10].keys())

ValueError: Column 'ground' not found in the DataFrame.